In [1]:
#measurment
dist=20

#file
filename="data/MST010 profiles C.TXT"
encoding="ansi"
sep="\t"

#result
interpolation_method = "linear" #possible: nearest,linear,cubic
target_resolution=1
tofile=True
filebase="data/MST010_profiles_C"
fig_dpi=600
basicviews=True


#advaced
cornerread=2 #how many points in each corner will be taken for plain estimation (e.g. 2x2 per corner if 2)


##
from IPython.display import display, HTML
op = "<div id='scratchoutput'></div><script>scratchoutput = document.getElementById('scratchoutput');</script>"
display(HTML(op))

In [2]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.interpolate as interpolate
from mpl_toolkits import mplot3d
import warnings

In [4]:
def add_to_div(jsobname,text,ele):
    js="<script type='text/javascript'>var para = document.createElement('"+ele+"');var node = document.createTextNode('"+text+"');para.appendChild(node);"+jsobname+".appendChild(para);</script>"
    #print(js)
    display(HTML(js))

class ScratchReader():
    def __init__(self,autorun=True,**kwargs):
        self.__dict__.update(kwargs)
        if not autorun:
            return
        if self.file is not None:
            self.read()
            
    def read(self,file=None):
        if file is not None:
            self.file=file
        if self.file is not None:
            add_to_div(self.output,"read file...","h3")
            self.df=pd.read_csv(self.file,encoding=encoding,sep=sep)
        else:
            warnings.warning("No file set")
sr = ScratchReader(file=filename,autorun=True,output='scratchoutput')

In [5]:
df = df.dropna(1)
data = df.values

newdata=[]
miny=+np.inf
maxy=-np.inf
for i in range(0,data.shape[1]-1,2):
    x = i/2*dist
    z = data[:,[1+i]].reshape(1,-1)[0]
    y = data[:,[0+i]].reshape(1,-1)[0]
    miny=min(miny,y.min())
    maxy=max(maxy,y.max())
    
    for j in range(len(z)):
        newdata.append((x,y[j],z[j]))

NameError: name 'df' is not defined

In [ ]:
def plot_xyz(xyz,res=1):
    xyz = np.array(xyz)

    max_values = np.amax(xyz,axis=0)
    min_values = np.amin(xyz,axis=0)
    xy_size = max_values-min_values
    xy_size_norm = xy_size/xy_size.max()
    
    X,Y = np.meshgrid(
        np.linspace(min_values[0],max_values[0], int(xy_size[0]/res)),
        np.linspace(min_values[1],max_values[1], int(xy_size[1]/res))
    )
    Z = interpolate.griddata((xyz[:,0], xyz[:,1]), xyz[:,2], (X,Y), method=interpolation_method)
    return X,Y,Z

X,Y,Z=plot_xyz(newdata[:],res=target_resolution)

mask = np.all(np.isnan(Z), axis=1)
X=X[~mask]
Y=Y[~mask]
Z=Z[~mask]
mask = np.all(np.isnan(Z), axis=0)
X=X[~mask]
Y=Y[~mask]
Z=Z[~mask]

In [ ]:
def get_flat(xyz_array,show=True):
    # create random data
    xs = xyz_array[:,0]
    ys = xyz_array[:,1]
    zs = xyz_array[:,2]


    # do fit
    tmp_A = []
    tmp_b = []
    for i in range(len(xs)):
        tmp_A.append([xs[i], ys[i], 1])
        tmp_b.append(zs[i])
    b = np.matrix(tmp_b).T
    A = np.matrix(tmp_A)
    fit = (A.T * A).I * A.T * b
    errors = b - A * fit
    residual = np.linalg.norm(errors)
    
    fit = fit.A1
    if not show:
        return(fit[0], fit[1], fit[2])
   
    # plot raw data
    plt.figure()
    ax = plt.subplot(111, projection='3d')
    ax.scatter(xs, ys, zs, color='b')

    # plot plane
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    Xf,Yf = np.meshgrid(np.arange(xlim[0], xlim[1]),
                      np.arange(ylim[0], ylim[1]))
    Zf = np.zeros(Xf.shape)
    for r in range(Xf.shape[0]):
        for c in range(Xf.shape[1]):
            Zf[r,c] = fit[0] * Xf[r,c] + fit[1] * Yf[r,c] + fit[2]
    ax.plot_wireframe(Xf,Yf,Zf, color='k')

    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    plt.show()
    return(fit[0], fit[1], fit[2])

    

def cornermean(a,d=4):
    return (np.nanmean(a[-d:,0:d]),np.nanmean(a[-d:,-d:]),
          np.nanmean(a[:d,:d]),np.nanmean(a[:d,-d:]))


def flat_data(X,Y,Z,points=np.array(cornermean(X,2)+cornermean(Y,2)+cornermean(Z,2)).reshape(3,4).transpose(),withPlane=True):
    xf,yf,cf = get_flat(cornerpoints,False)
    planeY=X*xf+Y*yf+cf
    Zn=Z-planeY
    return Zn,planeY

def remove_baseplaine(z,nerror=2,mean=None,std=None,std_error=None,val=np.nan):
    if mean is None:
        mean=np.nanmean(z)
    if std is None:
        std=np.nanstd(z)
    if std_error is None:
        std_error=std/np.sqrt(len(z))
#    print(mean,std,std_error,vari)
    nZ=np.copy(z)
    nZ[((nZ<mean+nerror*std_error)&(nZ>mean-nerror*std_error))]=val
    return nZ

cornerpoints=np.array(cornermean(X,cornerread)+cornermean(Y,cornerread)+cornermean(Z,cornerread)).reshape(3,4).transpose()
nZ,planeY = flat_data(X,Y,Z,cornerpoints)
nZ2 = remove_baseplaine(nZ,nerror=3,mean=0,val=0)

In [ ]:
%matplotlib inline
# set the colormap and centre the colorbar
import matplotlib.colors as colors
class MidpointNormalize(colors.Normalize):
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y), np.isnan(value))

cmap="RdYlGn"    
print("raw")
fig = plt.figure(figsize=xy_size_norm[:-1]*20)
plt.pcolormesh(X,Y,Z,cmap=cmap,norm=MidpointNormalize(midpoint=np.nanmean(Z),vmin=np.nanmin(Z), vmax=np.nanmax(Z)))
plt.colorbar(),plt.show()
print("baseplane")
fig = plt.figure(figsize=xy_size_norm[:-1]*20)
plt.pcolormesh(X,Y,planeY,cmap=cmap,norm=MidpointNormalize(midpoint=np.nanmean(planeY),vmin=np.nanmin(planeY), vmax=np.nanmax(planeY)))
plt.colorbar(),plt.show()
print("baseplane substarcted")
fig = plt.figure(figsize=xy_size_norm[:-1]*20)
plt.pcolormesh(X,Y,nZ,
               cmap=cmap,
               norm=MidpointNormalize(midpoint=0,vmin=np.nanmin(nZ), vmax=np.nanmax(nZ)))
plt.colorbar(),plt.show()
print("baseplane extracted")
fig = plt.figure(figsize=xy_size_norm[:-1]*20)
plt.pcolormesh(X,Y,nZ2,
               cmap=cmap,
               norm=MidpointNormalize(midpoint=0,vmin=np.nanmin(nZ2), vmax=np.nanmax(nZ2)))
plt.colorbar(),plt.show()


In [ ]:
print(1)

In [ ]:
%matplotlib inline
from matplotlib import cm
i=1
def plotsurface(X,Y,Z,a=45,b=-45,title="fig",save=False, figsize=(4,4),evenscaled=False,dpi=100):
    global i
    Z=np.nan_to_num(Z,True)
    fig = plt.figure(figsize=figsize,dpi=dpi)
    
    ax = plt.axes(projection='3d')
    #plt.axis('equal')
    #plt.axis([np.nanmin(X),np.nanmax(X)*2, np.nanmin(Y),np.nanmax(Y)*2])
    ax.view_init(a, b)
#    ax.axis('equal')
    my_col = cm.jet(Z/np.amax(Z))
    
    surf = ax.plot_surface(X, Y, Z, rstride=5, cstride=5, cmap = "RdYlGn",norm=MidpointNormalize(midpoint=0,vmin=np.nanmin(Z), vmax=np.nanmax(Z)),
                           linewidth=0, antialiased=False)
    fig.colorbar(surf, shrink=0.5, aspect=5)

    if evenscaled:
        max_range = np.array([X.max()-X.min(), Y.max()-Y.min(), Z.max()-Z.min()]).max()
        Xb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][0].flatten() + 0.5*(X.max()+X.min())
        Yb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][1].flatten() + 0.5*(Y.max()+Y.min())
        Zb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][2].flatten() + 0.5*(Z.max()+Z.min())
        # Comment or uncomment following both lines to test the fake bounding box:
        for xb, yb, zb in zip(Xb, Yb, Zb):
            ax.plot([xb], [yb], [zb], 'w')

    if save:
        fig.savefig('title_'+str(i))
        i += 1
    fig.show()


plotsurface(X,Y,nZ2,45,-30,dpi=2000,evenscaled=True,save=True)
plotsurface(X,Y,nZ2,45,-30,dpi=1000,save=True)
#plotsurface(45,-10,)
#plotsurface(45,0,)
#plotsurface(0,90,)
#plotsurface(65,170,)
#plotsurface(45,-10,figsize=(4,res[0]))
#plotsurface(45,0,figsize=(4,res[0]))
#plotsurface(0,90,figsize=res)
#plotsurface(65,170,figsize=(4,res[0]))

In [30]:

x=np.array([0,1,2,3])
y=np.array([0,1])
Z=np.array([[0,0,0,0],[0,0,0,0]])
np.repeat(x,Z.shape[0]).reshape(Z.shape[1],Z.shape[0]).transpose()
np.repeat(y,Z.shape[1]).reshape(Z.shape)

array([[0, 0, 0, 0],
       [1, 1, 1, 1]])

In [82]:
a = np.zeros((15,11))
a[1][1]=-1
a[0][2]=2
a[2][0]=3
a
amin = np.argmin(a)
y = amin%a.shape[1]
x = int((amin-y)/a.shape[1])
x,y,amin,y*a.shape[0],a[x][y]

(1, 1, 12, 15, -1.0)

In [72]:
a.flatten()

array([ 0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  3.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [73]:
a

array([[ 0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])